In [1]:
from sklearn.datasets import load_breast_cancer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F


class mydata(Dataset):
    def __init__(self, features, targets):
        self.x_data = torch.tensor(features, dtype=torch.float32)
        self.y_data = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)


class BinaryModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BinaryModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return F.sigmoid(self.linear(x))
        # return self.linear(x)


data = load_breast_cancer()
# print(len(data.data[0]))
# print(data.target)

x_data = data.data
y_data = data.target

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.1, random_state=42
)

train_data = mydata(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

test_data = mydata(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

print(len(x_data[0]))

# input_dim = len(x_data[0])
# output_dim = 1

# Model setup
input_dim = x_train.shape[1]
output_dim = 1
model = BinaryModel(input_dim, output_dim)

# criterion = torch.nn.MSELoss()
criterion = torch.nn.BCELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 1001
loss_list = []
accuracy = []

for epoch in range(epochs):
    loss_sum = 0
    for inputs, targets in train_loader:
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    if epoch % 100 == 0:
        print(f"epoch : {epoch}, loss : { loss_sum / len(train_loader)}")

30


/home/lim/Software/anaconda3/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch : 0, loss : 1961.372413635254
epoch : 100, loss : 1981.25
epoch : 200, loss : 1981.25
epoch : 300, loss : 1981.25
epoch : 400, loss : 1981.25
epoch : 500, loss : 1981.25
epoch : 600, loss : 1981.25
epoch : 700, loss : 1981.25
epoch : 800, loss : 1981.25
epoch : 900, loss : 1981.25
epoch : 1000, loss : 1981.25


In [3]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


# Dataset class
class MyData(Dataset):
    def __init__(self, features, targets):
        self.x_data = torch.tensor(features, dtype=torch.float32)
        self.y_data = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)


# Model class
class BinaryModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BinaryModel, self).__init__()
        self.linear1 = torch.nn.Linear(30, 16)
        self.linear2 = torch.nn.Linear(16, 8)
        self.linear3 = torch.nn.Linear(8, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.sigmoid(x)

        x = self.linear2(x)
        x = self.sigmoid(x)

        x = self.linear3(x)
        x = self.sigmoid(x)  # Sigmoid to produce output in the range [0, 1]
        return x

    # Linear output; sigmoid applied later for better numerical stability


# Load the dataset
data = load_breast_cancer()
x_data = data.data
y_data = data.target

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.1, random_state=42
)

# Create datasets and dataloaders
train_data = MyData(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

test_data = MyData(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

# Model setup
input_dim = x_train.shape[1]
output_dim = 1
model = BinaryModel(input_dim, output_dim)

# Define loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()  # More suitable for binary classification
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 1001
for epoch in range(epochs):
    loss_sum = 0
    correct = 0
    total = 0
    for inputs, targets in train_loader:
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

        # Calculate accuracy
        predicted = torch.sigmoid(y_pred)  # Apply sigmoid to get probabilities
        predicted = (predicted > 0.5).float()  # Convert probabilities to 0 or 1
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    accuracy = correct / total
    if epoch % 100 == 0:
        print(
            f"Epoch: {epoch}, Loss: {loss_sum / len(train_loader)}, Accuracy: {accuracy * 100:.2f}%"
        )

print("Training complete.")

Epoch: 0, Loss: 0.6663275100290775, Accuracy: 61.91%
Epoch: 100, Loss: 0.6648819707334042, Accuracy: 61.91%
Epoch: 200, Loss: 0.6645689755678177, Accuracy: 61.91%
Epoch: 300, Loss: 0.6643445044755936, Accuracy: 61.91%
Epoch: 400, Loss: 0.6639577448368073, Accuracy: 61.91%
Epoch: 500, Loss: 0.6633362136781216, Accuracy: 61.91%
Epoch: 600, Loss: 0.6625021509826183, Accuracy: 61.91%
Epoch: 700, Loss: 0.6615314595401287, Accuracy: 61.91%
Epoch: 800, Loss: 0.6600909195840359, Accuracy: 61.91%
Epoch: 900, Loss: 0.6583434194326401, Accuracy: 61.91%
Epoch: 1000, Loss: 0.655857227742672, Accuracy: 61.91%
Training complete.


In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


class mydata(Dataset):
    def __init__(self, features, targets):
        self.x_data = torch.tensor(features, dtype=torch.float32)
        self.y_data = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)


class BinaryModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BinaryModel, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, 8)
        self.linear2 = torch.nn.Linear(8, 4)
        self.linear3 = torch.nn.Linear(4, 1)

    def forward(self, x):
        out = F.relu(self.linear1(x))
        out = F.relu(self.linear2(out))
        out = torch.sigmoid(self.linear3(out))  # 仅在最后一层使用Sigmoid
        return out


# Load and split data
data = load_breast_cancer()
x_data = data.data
y_data = data.target

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.1, random_state=42
)

train_data = mydata(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

test_data = mydata(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

input_dim = x_data.shape[1]
output_dim = 1
model = BinaryModel(input_dim, output_dim)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 101

for epoch in range(epochs):
    loss_sum = 0
    for batch_idx, data in enumerate(train_loader):
        inputs, targets = data
        y_pred = model(inputs)

        loss = criterion(y_pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    if epoch % 10 == 0:  # 每10轮打印一次
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, targets = data
                outputs = model(inputs)
                predicted = outputs.round()  # 四舍五入到0或1
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        accuracy = 100 * correct / total
        print(
            f"Epoch: {epoch}, Loss: {loss_sum / len(train_loader):.4f}, Accuracy: {accuracy:.2f}%"
        )

Epoch: 0, Loss: 0.7590, Accuracy: 29.82%
Epoch: 10, Loss: 0.6935, Accuracy: 70.18%
Epoch: 20, Loss: 0.6776, Accuracy: 70.18%
Epoch: 30, Loss: 0.6705, Accuracy: 70.18%
Epoch: 40, Loss: 0.6673, Accuracy: 70.18%
Epoch: 50, Loss: 0.6658, Accuracy: 70.18%
Epoch: 60, Loss: 0.6651, Accuracy: 70.18%
Epoch: 70, Loss: 0.6648, Accuracy: 70.18%
Epoch: 80, Loss: 0.6646, Accuracy: 70.18%
Epoch: 90, Loss: 0.6646, Accuracy: 70.18%
Epoch: 100, Loss: 0.6645, Accuracy: 70.18%
